# Download Quads from a Mosiac Using Basemaps API
---

## Overview ##
---
The purpose of this tutorial is to provide a user-friendly way to use Planet's Basemaps API to download all quads to a folder in their directory.

## Table of Contents

---

* **[How to Get Started](#how-to-get-started)**
    * Imports and Packages
    
* **[Setup Session Using Basemaps API](#setup-session)**
    * Create session to access API
    * Send request and check status code
    
* **[Access Mosaic Metadata](#mosaic-meta)**
    * Further examine the json object
    * Get id and bbox values from mosaic 
    
* **[Search for Mosaic Quads Using Area of Interest](#search-quads)**
    * Setup request for quads to basemaps API
    * Quads metadata
    
* **[Download and Save Quads to Directory](#save-quads)**
    * Iterate over quad download links and save as individual files

* **[Conclusion](#conclusion)**

<a name="how-to-get-started"></a> 
## How To Get Started
---
Before we jump into downloading the quads, let's quickly review the required packages for utilizing this tutorial.

### Imports

* requests
* os
* json
* Your Planet API Key
* urllib.request

<a name="setup-session"></a> 
## Setup Session Using Basemaps API
---
The first step is to setup a session to access the basemaps API. The basemaps API can be used to download full mosaics, or in our case, to download individual parts of the mosaic known as quads. 

### Create Session to Access API
---
In order to access the basemaps API, we need a Planet API key for successful authentication. In this case, the API key is saved to a config file and imported into the python script, but feel free to use your API key directly in the notebook if you are following along.

In [ ]:
import os
import requests

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'PASTE_API_KEY_HERE')

# Setup Planet Data API base URL
URL = "https://api.planet.com/basemaps/v1/mosaics"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

### Send Request and Check Status Code
---
When sending the request, we will provide one parameter. Assuming we know the name of the mosaic, we can include those details in the request to access the corresponding metadata. After sending the request, let's check the status code to ensure it was successful.

In [ ]:
#set params for search using name of mosaic
parameters = {
    "name__is" :"nsw_ps_1month_date_ramp_L15_mosaic" # <= customize to your use case
}
#make get request to access mosaic from basemaps API
res = session.get(API_URL, params = parameters)
#response status code
print(res.status_code)

<a name="mosaic-meta"></a> 
## Access Mosaic MetaData
---
The metadata for each mosaic is necessary to access the quads. Specifically, we need to extract the mosaic id and bbox to search for the quads within our area of interest. In this tutorial we are using the AOI from the entire mosaic to download all quads, but those values can be customized for your use case.


### Further Examine the JSON Object


In [ ]:
import json

#print metadata for mosaic
mosaic = res.json()
print(json.dumps(mosaic, indent=2))

### Get ID and Bbox Values from Mosaic

In [ ]:
#get id
mosaic_id = mosaic['mosaics'][0]['id']
#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']
#converting bbox to string for search params
string_bbox = ','.join(map(str, mosaic_bbox))

print('Mosaic id: '+ mosaic_id)
print('Mosaic bbox: '+ string_bbox)

<a name="search-quads"></a> 
## Search for Mosaic Quads using AOI
---
Now that we have the required metadata, we can send a request to the API for the mosaic's quads. Each quad comes with a download link that we use to save them to our local directory. 

### Setup Request for Quads to Basemaps API
---
We will provide two parameters, a string of bbox values and minimal set to "True". The bbox values are required to send the request. The minimal parameter is optional, but we included it because it hides the metadata we don't need for this use case. Lastly, we include the mosaic id in the request url to access quads for this specific mosaic.

In [ ]:
#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}
#accessing quads using metadata from mosaic
quads_url = "{}/{}/quads".format(API_URL, mosaic_id)
res = session.get(quads_url, params=search_parameters, stream=True)
print(res.status_code)

### Quads Metadata
---
The quad response object contains the download link and bbox values for each portion of the grid. Each quad also has a unique identifier which we will use to name the quads as we save them to our local directory.

In [ ]:
quads = res.json()
items = quads['items']
#printing an example of quad metadata
print(json.dumps(items[0], indent=2))

<a name="save-quads"></a> 

## Download Quads to Local Directory
---
The final step to this tutorial is taking the download link from the metadata and saving those files to your local directory by id number. We do this by iterating over the json object and using urllib to retrieve the file via the download link.

In [ ]:
from urllib.request import urlopen
from shutil import copyfileobj

# Iterate over quad download links and saving to folder by id.
# This may take quite some time. 
for i in items:
    link = i['_links']['download']
    # print (link)
    name = i['id']
    name = name + '.tiff'

    with urlopen(link) as in_stream, open(name, 'wb') as out_file:
        copyfileobj(in_stream, out_file)
        print (name)

## Conclusion
---
After a few simple steps, we now have a folder full of quads within our local directory. Check out our [Planet Basemaps API Documentation](https://developers.planet.com/docs/basemaps/reference/) to learn more. If you'd like to access the full .py script, follow this [link](download_quad.py).

Questions or comments about this guide? Join the conversation at [Planet Community](https://community.planet.com/).